In [1]:
# !pip install -q pydicom albumentations focal_loss torchsampler timm lion-pytorch

In [2]:
!nvidia-smi

Tue Oct 17 14:00:34 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.04              Driver Version: 536.25       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000               On  | 00000000:07:00.0  On |                  Off |
| 41%   41C    P8              16W / 140W |    784MiB / 16376MiB |     15%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# 必要なパッケージのimportを記述しましょう
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import random
import warnings
warnings.simplefilter('ignore')
import wandb

from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
import cv2
import pydicom

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from timm import create_model, list_models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from timm.scheduler import CosineLRScheduler
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler
from torch.nn import BCEWithLogitsLoss, NLLLoss2d, MSELoss, BCELoss, CrossEntropyLoss
from torch.nn import Flatten
from torch.utils import data as data
INPUT_PATH = "../../input/"
IMAGE_PATH = "../../input/train_images/"
device= "cuda" if torch.cuda.is_available() else 'cpu'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:
list_models('*maxvit*')

['maxvit_base_tf_224',
 'maxvit_base_tf_384',
 'maxvit_base_tf_512',
 'maxvit_large_tf_224',
 'maxvit_large_tf_384',
 'maxvit_large_tf_512',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_base_rw_224',
 'maxvit_rmlp_base_rw_384',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_tf_224',
 'maxvit_small_tf_384',
 'maxvit_small_tf_512',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_tiny_tf_224',
 'maxvit_tiny_tf_384',
 'maxvit_tiny_tf_512',
 'maxvit_xlarge_tf_224',
 'maxvit_xlarge_tf_384',
 'maxvit_xlarge_tf_512']

In [5]:
device

'cuda'

In [6]:
os.cpu_count()

12

In [7]:
train = pd.read_parquet('slice_train_folds.parquet').reset_index(drop=True)
train['path'] = '../../input/'+train['path']
# train['path'] = train['path'].apply(lambda x: 'train_png'+x[12:-3]+'png')
train.head()

,InstanceNumber,ImagePositionPatient,path,RescaleType,series_id,patient_id,stratify,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,fold
0,66,1832.0,../../input/train_images/49954/41479/66.dcm,nan,41479,49954,1010.01001001000,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1
1,78,1822.4,../../input/train_images/49954/41479/78.dcm,nan,41479,49954,1010.01001001000,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1
2,90,1812.8,../../input/train_images/49954/41479/90.dcm,nan,41479,49954,1010.01001001000,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1
3,102,1803.2,../../input/train_images/49954/41479/102.dcm,nan,41479,49954,1010.01001001000,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1
4,115,1792.8,../../input/train_images/49954/41479/115.dcm,nan,41479,49954,1010.01001001000,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1


In [8]:
train.columns

Index(['InstanceNumber', 'ImagePositionPatient', 'path', 'RescaleType',
       'series_id', 'patient_id', 'stratify', 'bowel_healthy', 'bowel_injury',
       'extravasation_healthy', 'extravasation_injury', 'kidney_healthy',
       'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 'liver_high',
       'spleen_healthy', 'spleen_low', 'spleen_high', 'any_injury', 'fold'],
      dtype='object')

In [9]:
tmp = train.copy()

tmp = pd.DataFrame(tmp.groupby('series_id')['InstanceNumber'].apply(list)).reset_index()

train = train[[
       'series_id', 'patient_id', 'bowel_healthy', 'bowel_injury',
       'extravasation_healthy', 'extravasation_injury', 'kidney_healthy',
       'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 'liver_high',
       'spleen_healthy', 'spleen_low', 'spleen_high', 'any_injury', 'fold']].drop_duplicates()

train = train.merge(tmp, on='series_id', how='left')

In [10]:
# 学習・推論時に用いるパラメータなどを指定しましょう(事前に実装するのは難しいので、実装を全て終えた後に確認でも可)
CFG = {
    # パラメータ
    'seed' : 42,
    'verbose' : 1, 
    'optimizer' : 'AdamW', 
    'train_batch_size' : 2,
    'val_batch_size' : 2, 
    'resize1': 224, 
    'resize2': 224, 
    'epoch' : 15, 
    'lr' :5e-5,
    # 'lr' :1e-5, 
    'fold' : 4, 
    'use_fold': [1,2,3], 
    'use_amp' : True, 
    'debug' : False,  # デバッグ用で動作だけ確認したいときはTrue(データ量を削減して実行)
    'data_root' : IMAGE_PATH, 
    "model" : "maxvit_tiny_tf_224", 
    # "model" : "tf_efficientnetv2_s",
    # "model" : "tf_efficientnetv2_l",
    # "model" : "swinv2_base_window16_256",
    # "model" : "tf_efficientnet_b0_ns",
    "weight" : False, 
    "slice_size" : 64, 
    "acc_step" : 4,
    "start_epoch" : 0
}

device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
# 実験の再現性を確保するため、各処理での乱数シードを固定しましょう(事前に実装するのは難しいので、実装を全て終えた後に確認でも可)
def seed_everything(seed):
    # 処理
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [12]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift

    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2    
    
    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)

    return pixel_array

def crop_image(img, show=False):
    tmp = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 画像を2値化。ピクセルの値が1より大きい場合は255(白)、それ以外の場合は0(黒)
    bin_pixels = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)[1]
   
    # Make contours around the binarized image, keep only the largest contour
    # 2値画像から輪郭を抽出。cv2.RETR_EXTERNALはもっとも外側絵の輪郭のみを抽出。
    contours, _ = cv2.findContours(bin_pixels, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # もしかしたらcountersがNoneの場合もあり得るので、エラー処理
    if not contours:
        return tmp 
    
    # 最大の輪郭を選択
    contour = max(contours, key=cv2.contourArea)

    # Create a mask from the largest contour
    # マスクを作成。
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
   
    # Use bitwise_and to get masked part of the original image
    # 
    out = cv2.bitwise_and(img, mask)
    
    # get bounding box of contour
    y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
    x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
    
    x1 = int(0.99 * x1)
    x2 = int(1.01 * x2)
    y1 = int(0.99 * y1)
    y2 = int(1.01 * y2)
    
    if show:
        plt.imshow(out[y1:y2, x1:x2], cmap="gray") ; 
    
    img = out[y1:y2, x1:x2]

    return img

def get_img(path):
    # 処理
    dicom = pydicom.dcmread(path)
    pp = dicom.ImagePositionPatient[2]
    img = standardize_pixel_array(dicom)
    img = (img - img.min()) / (img.max() - img.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img
    img = np.dstack((img, img, img))
    img = (img * 255).astype(np.uint8)
    img = crop_image(img)
    if len(img.shape) == 3:
        img = img[:,:,-1]

    return img, pp

# def get_img(path):
#     img = cv2.imread(path)
#     img = cv2.split(img)[0]
#     return img

In [13]:
# 学習用のデータ変換処理を定義しましょう
def get_train_transforms():
    return albu.Compose([
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.5),
            albu.OneOf([
                albu.GaussianBlur(),
                albu.MotionBlur(),
            ], p=0.3),
            albu.OneOf([
                albu.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
                albu.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
                albu.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
            ], p=0.3),
            albu.Resize(CFG['resize1'], CFG['resize2'], interpolation=cv2.INTER_NEAREST),
            # albu.CenterCrop(int(CFG['resize1']*0.9), int(CFG['resize2']*0.9)),
            ], p=1.0)
  
# 検証用のデータ変換処理を定義しましょう
def get_valid_transforms():
    return albu.Compose([
        # 処理
        albu.Resize(height=CFG['resize1'], width=CFG['resize2'], interpolation=cv2.INTER_NEAREST),
        # albu.CenterCrop(int(CFG['resize1']*0.9), int(CFG['resize2']*0.9)),
        ], p=1.)

In [14]:
# # 訓練時の画像前処理の動作を確認
# # 1. サンプルとして1つの画像を読み込みましょう
# image = get_img("../../input/train_png/10004/21057/1007.png")
# print(image.shape)
# # 2. 1で読み込んだ画像を表示しましょう
# fig, (axL, axR) = plt.subplots(ncols=2, figsize=(20,6))

# axL.imshow(image)
# # 3. get_train_transforms()で定義した変換で読み込んだ画像を変換しましょう
# transformed = get_train_transforms()
# transformed = transformed(image=image)
# transformed_image = transformed["image"]

# # 4. 3で変換した画像を表示しましょう
# transformed_image = transformed_image.permute(1, 2, 0)

# axR.imshow(transformed_image)
# #plt.imshow(transformed_image)

In [15]:
imgs = np.load('../../../input/train_npy_224_64/'+str(49954)+'_'+str(41479)+'.npy')

In [16]:
imgs.shape

(224, 224, 64)

In [17]:
# for i in range(CFG['slice_size']):
#     plt.imshow(imgs[:,:,i])
#     plt.show()


In [18]:
train.head()

,series_id,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,fold,InstanceNumber
0,41479,49954,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,1,"[66, 78, 90, 102, 115, 127, 139, 151, 163, 175..."
1,59269,50737,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,0,"[1, 14, 26, 39, 51, 64, 76, 89, 101, 114, 126,..."
2,29738,55903,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,2,"[41, 42, 44, 45, 46, 48, 49, 51, 52, 53, 55, 5..."
3,5363,55903,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,2,"[154, 152, 149, 147, 144, 142, 139, 137, 135, ..."
4,62342,51078,1,0,1,0.0,1,0,0,1,0,0,1,0,0,0,3,"[1, 3, 6, 8, 11, 13, 16, 18, 21, 23, 26, 28, 3..."


In [19]:
# 今回の学習に用いるデータセットを定義しましょう
class BreastDataset(Dataset):
    """
    RSNA Breast Cancer Detection のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    df : DataFrame
        正解ラベルの入ったデータフレーム
    data_root : str
        画像のパス
    transform : object
        前処理クラス
    output_label : bool
        正解ラベルがわかっている場合はTrue
    """

    def __init__(self,
                 df,
                 transforms=None, 
                 output_label=True):
        # 処理
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.output_label = output_label

    def __len__(self):
        '''画像の枚数を返す'''
        return self.df.shape[0]
        
    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''
        # 処理
        row = self.df.loc[index]
        # slice_list = row['InstanceNumber']
        # first = None
        # last = None
        # imgs = []
        # for s in slice_list:
        #     p = '../../../input/train_images/'+str(row['patient_id'])+'/'+str(row['series_id'])+'/'+str(s)+'.dcm'
        #     img, pp = get_img(p)            
        #     img = cv2.resize(img, (CFG['resize1'], CFG['resize2']))
        #     imgs.append(img)
        # # imgs = np.array(imgs)
        # imgs = np.stack(imgs, axis=2)

        imgs = np.load('../../../input/train_npy_224_64/'+str(row['patient_id'])+'_'+str(row['series_id'])+'.npy')
        
        imgs = self.transforms(image=imgs)
        imgs = imgs["image"]
        imgs = np.transpose(imgs, (2, 0, 1))
        # 正解ラベルがある場合は、変換後の画像とラベルを両方返す
        if self.output_label == True:
            labels = row.values[2:-3]
            bowel = np.argmax(labels[0:2], keepdims = False)
            extravasation = np.argmax(labels[2:4], keepdims = False)
            kidney = np.argmax(labels[4:7], keepdims = False)
            liver = np.argmax(labels[7:10], keepdims = False)
            spleen = np.argmax(labels[10:], keepdims = False)
            target = np.array([bowel, extravasation, kidney, liver, spleen])
            
            return torch.from_numpy(imgs).float(), torch.from_numpy(target).float()
        else:
            return torch.from_numpy(imgs).float()

In [20]:
import torch.nn as nn
from itertools import repeat

class SpatialDropout(nn.Module):
    def __init__(self, drop=0.5):
        super(SpatialDropout, self).__init__()
        self.drop = drop
        
    def forward(self, inputs, noise_shape=None):
        """
        @param: inputs, tensor
        @param: noise_shape, tuple
        """
        outputs = inputs.clone()
        if noise_shape is None:
            noise_shape = (inputs.shape[0], *repeat(1, inputs.dim()-2), inputs.shape[-1]) 
        
        self.noise_shape = noise_shape
        if not self.training or self.drop == 0:
            return inputs
        else:
            noises = self._make_noises(inputs)
            if self.drop == 1:
                noises.fill_(0.0)
            else:
                noises.bernoulli_(1 - self.drop).div_(1 - self.drop)
            noises = noises.expand_as(inputs)    
            outputs.mul_(noises)
            return outputs
            
    def _make_noises(self, inputs):
        return inputs.new().resize_(self.noise_shape)


import torch
from torch import nn
import torch.nn.functional as F

from typing import Dict, Optional
 
import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor


    
class MLPAttentionNetwork(nn.Module):
 
    def __init__(self, hidden_dim, attention_dim=None):
        super(MLPAttentionNetwork, self).__init__()
 
        self.hidden_dim = hidden_dim
        self.attention_dim = attention_dim
        if self.attention_dim is None:
            self.attention_dim = self.hidden_dim
        # W * x + b
        self.proj_w = nn.Linear(self.hidden_dim, self.attention_dim, bias=True)
        # v.T
        self.proj_v = nn.Linear(self.attention_dim, 1, bias=False)
 
    def forward(self, x):
        """
        :param x: seq_len, batch_size, hidden_dim
        :return: batch_size * seq_len, batch_size * hidden_dim
        """
        # print(f"x shape:{x.shape}")
        batch_size, seq_len, _ = x.size()
        # flat_inputs = x.reshape(-1, self.hidden_dim) # (batch_size*seq_len, hidden_dim)
        # print(f"flat_inputs shape:{flat_inputs.shape}")
        
        H = torch.tanh(self.proj_w(x)) # (batch_size, seq_len, hidden_dim)
        # print(f"H shape:{H.shape}")
        
        att_scores = torch.softmax(self.proj_v(H),axis=1) # (batch_size, seq_len)
        # print(f"att_scores shape:{att_scores.shape}")
        
        attn_x = (x * att_scores).sum(1) # (batch_size, hidden_dim)
        # print(f"attn_x shape:{attn_x.shape}")
        return attn_x

In [21]:
class RSNAClassifier(nn.Module):
    def __init__(self, model_arch, hidden_dim=256, seq_len=32, pretrained=False):
        super().__init__()
        self.seq_len = seq_len
        self.model = create_model(model_arch, in_chans=1, pretrained=pretrained)

        if 'efficientnet' in CFG['model']:
            cnn_feature = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
        elif "res" in CFG['model']:
            cnn_feature = self.model.fc.in_features
            self.model.global_pool = nn.Identity()
            self.model.fc = nn.Identity()
            self.pooling = nn.AdaptiveAvgPool2d(1)
        elif "convnext" in CFG['model']:
            cnn_feature = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        elif "fastvit" in CFG['model']:
            cnn_feature = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        elif "caformer" in CFG['model']:
            cnn_feature = self.model.head.fc.fc2.in_features
            self.model.head.fc.fc2 = nn.Identity()
        elif "eca_nfnet" in CFG['model']: 
            cnn_feature = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        elif "efficientvit_m" in CFG['model']:
            cnn_feature = self.model.head.linear.in_features
            self.model.head.linear = nn.Identity()
        elif 'efficientvit_b' in CFG['model']:
            cnn_feature = self.model.head.classifier[4].in_features
            self.model.head.classifier[4] = nn.Identity()
        elif 'swinv2' in CFG['model']:
            cnn_feature = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        elif 'eva02' in CFG['model']:
            cnn_feature = self.model.head.in_features
            self.model.head = nn.Identity()
        elif 'maxvit' in CFG['model']:
            cnn_feature = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()

        freeze_len = 0
        for i,(name, param) in enumerate(list(self.model.named_parameters())):
            freeze_len = i
        freeze_len = int(freeze_len / 5)
        for i,(name, param) in enumerate(list(self.model.named_parameters())):
            if i < freeze_len:
                param.requires_grad = False

        self.spatialdropout = SpatialDropout(0.1)
        self.gru = nn.GRU(cnn_feature, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
        self.mlp_attention_layer = MLPAttentionNetwork(2 * hidden_dim)

        self.bowel = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1)
        )
        
        self.extravasation = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1)
        )
        
        self.kidney = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 3)
        )
        
        self.liver = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 3)
        )
        
        self.spleen = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 3)
        )

    def forward(self, x): # (B, seq_len, H, W)
        bs = x.size(0) 
        x = x.reshape(bs*self.seq_len, 1, x.size(2), x.size(3)) # (B*seq_len, 1, H, W) : 1チャネルの画像モデルに通すことができる
        features = self.model(x)   
        if "res" in CFG['model']:                             
            features = self.pooling(features).view(bs*self.seq_len, -1) # (B*seq_len, cnn_feature)
        features = self.spatialdropout(features)                # (B*seq_len, cnn_feature)
        # print(features.shape)
        features = features.reshape(bs, self.seq_len, -1)       # (B, seq_len, cnn_feature)
        features, _ = self.gru(features)                        # (B, seq_len, hidden_dim*2)
        atten_out = self.mlp_attention_layer(features)          # (B, hidden_dim*2)

        bowel = self.bowel(atten_out)
        extravsation = self.extravasation(atten_out)
        kidney = self.kidney(atten_out)
        liver = self.liver(atten_out)
        spleen = self.spleen(atten_out)
        return bowel, extravsation, kidney, liver, spleen

In [22]:
# model = RSNAClassifier(CFG['model'], seq_len=CFG['slice_size'], pretrained=True)
# inp = torch.randn(2,CFG['slice_size'],CFG['resize1'],CFG['resize2'])
# out = model(inp)
# out

In [23]:
import numpy as np
import pandas as pd
import pandas.api.types
import sklearn.metrics


class ParticipantVisibleError(Exception):
    pass


def normalize_probabilities_to_one(df: pd.DataFrame, group_columns: list) -> pd.DataFrame:
    # Normalize the sum of each row's probabilities to 100%.
    # 0.75, 0.75 => 0.5, 0.5
    # 0.1, 0.1 => 0.5, 0.5
    row_totals = df[group_columns].sum(axis=1)
    if row_totals.min() == 0:
        raise ParticipantVisibleError('All rows must contain at least one non-zero prediction')
    for col in group_columns:
        df[col] /= row_totals
    return df


def score(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    '''
    Pseudocode:
    1. For every label group (liver, bowel, etc):
        - Normalize the sum of each row's probabilities to 100%.
        - Calculate the sample weighted log loss.
    2. Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    3. Calculate the sample weighted log loss for the new label group
    4. Return the average of all of the label group log losses as the final score.
    '''
    weight_cfg = {
        'bowel_weight': 2.0, 
        'extravsation_weight' : 6.0, 
        'kidney_weight' : [1.0, 2.0, 4.0],
        'liver_weight' : [1.0, 2.0, 4.0],
        'spleen_weight' : [1.0, 2.0, 4.0],
        'any_injury_weight' : 6.0
    }

    # Calculate the label group log losses
    binary_targets = ['bowel', 'extravasation']
    triple_level_targets = ['kidney', 'liver', 'spleen']
    all_target_categories = binary_targets + triple_level_targets

    label_group_losses = []
    for category in all_target_categories:
        if category in binary_targets:
            col_group = [f'{category}_healthy', f'{category}_injury']
        else:
            col_group = [f'{category}_healthy', f'{category}_low', f'{category}_high']

        solution = normalize_probabilities_to_one(solution, col_group)

        for col in col_group:
            if col not in submission.columns:
                raise ParticipantVisibleError(f'Missing submission column {col}')
        submission = normalize_probabilities_to_one(submission, col_group)
        label_group_losses.append(
            sklearn.metrics.log_loss(
                y_true=solution[col_group].values,
                y_pred=submission[col_group].values,
                sample_weight=solution[f'{category}_weight'].values
            )
        )

    # Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    healthy_cols = [x + '_healthy' for x in all_target_categories]
    any_injury_labels = (1 - solution[healthy_cols]).max(axis=1)
    any_injury_predictions = (1 - submission[healthy_cols]).max(axis=1)
    any_injury_loss = sklearn.metrics.log_loss(
        y_true=any_injury_labels.values,
        y_pred=any_injury_predictions.values,
        sample_weight=solution['any_injury_weight'].values
    )

    label_group_losses.append(any_injury_loss)
    return np.mean(label_group_losses)

In [24]:
# データローダーを定義しましょう
def prepare_dataloader(df,
                       fold,
                       data_root=CFG['data_root']):
    
    train_ = df[df['fold']!=fold].reset_index(drop=True)
    valid_ = df[df['fold']==fold].reset_index(drop=True)

    # 学習用のデータセットを読み込む
    train_ds = BreastDataset(train_, 
                              transforms=get_train_transforms(), 
                              output_label=True)
    
    # 検証用のデータセットを読み込む
    valid_ds = BreastDataset(valid_, 
                              transforms=get_valid_transforms(), 
                              output_label=True)
    
    # train_loaderを作成
    train_loader = DataLoader(train_ds, 
                              batch_size=CFG['train_batch_size'], 
                              num_workers=os.cpu_count(), 
                              pin_memory=True, 
                              drop_last=True, 
                              shuffle=True)
    
    # val_loaderを作成
    val_loader = DataLoader(valid_ds, 
                            batch_size=CFG['val_batch_size'], 
                            num_workers=os.cpu_count(), 
                            pin_memory=True, 
                            drop_last=True)

    return train_loader, val_loader

In [25]:
class GradualWarmupSchedulerV3(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV3, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [26]:
def training(train):
    
    seed_everything(CFG['seed'])
    
    # 1. データを学習と検証に分割し、学習用データを使って学習する
    for fold in range(4):
        if fold in CFG["use_fold"]:
            wandb.init(
                project='RSNA-ATD',
                name=CFG['model']+'25D_training',
                group=CFG['model']+'25D_training',
                job_type="train",
                anonymous=None,
                reinit=True,
            )
            print('Fold {}'.format(fold))
            print('-------------------------------------------------')

            model = RSNAClassifier(CFG['model'], seq_len=CFG['slice_size'], pretrained=True)
            if CFG['weight']:
                model.load_state_dict(torch.load('./cls_model/'+CFG['model']+'_fold_{}'.format(fold)))
            model.train()
            model.to(device)


            train_dataloader, val_dataloader = prepare_dataloader(train, fold, data_root=CFG['data_root'])
            
            bce_b = BCEWithLogitsLoss(pos_weight = torch.tensor([2.0]).to('cuda'))
            bce_e = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([6.0]).to('cuda'))
            cce = CrossEntropyLoss(weight = torch.tensor([1.0, 2.0, 4.0]).to('cuda'))
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'])
            scheduler = CosineLRScheduler(optimizer, CFG['epoch'], 
                        lr_min=CFG['lr']*0.1, 
                        warmup_t=int(CFG['epoch']/5), 
                        warmup_lr_init=CFG['lr']*0.1,
                        warmup_prefix=True
                        )
            # scheduler = CosineAnnealingLR(optimizer, 
            #             T_max=CFG['epoch']-3, 
            #             eta_min=CFG['lr']*0.1, 
            #             last_epoch=-1)
            # scheduler_warmup = GradualWarmupSchedulerV3(optimizer, 
            #             multiplier=10, 
            #             total_epoch=1, 
            #             after_scheduler=scheduler)
            
            torch.backends.cudnn.benchmark = True
            best_v_loss = 10.0
            for epoch in range(CFG['epoch']):
                if CFG['start_epoch'] <= epoch:
                    t_loss = 0.0
                    v_loss = 0.0
                    bowel_labels = []
                    extravsation_labels = []
                    kidney_labels = []
                    liver_labels = []
                    spleen_labels = []
                    
                    bowel_preds = []
                    extravsation_preds = []
                    kidney_preds = []
                    liver_preds = []
                    spleen_preds = []

                    bowel_pred_l = np.array([])
                    extravsation_pred_l = np.array([])
                    kidney_list_l = []
                    liver_list_l = []
                    spleen_list_l = []

                    dataloaders_dict = {'train':train_dataloader, 'val':val_dataloader}
                
                    # 2. 学習時・検証時のロスと正解率を出力しましょう
                    for phase in ['train', 'val']:
                        if phase == 'train':
                            model.train()  
                            # enabled:AMPを使用するかどうか
                            # https://tawara.hatenablog.com/entry/2021/05/31/220936
                            scaler = GradScaler(enabled=CFG['use_amp']) 
                        else:
                            model.eval()   

                        epoch_loss = 0.0  

                        # データローダーからミニバッチを取り出すループ
                        optimizer.zero_grad()
                        progress_bar = tqdm(dataloaders_dict[phase])
                        for i, (inputs, labels) in enumerate(progress_bar):

                            # GPUが使えるならGPUにデータを送る
                            inputs = inputs.to(device, non_blocking=True)
                            inputs = inputs.float()
                            bowel_label = labels[:, 0].float().to(device, non_blocking=True)
                            extravsation_label = labels[:, 1].float().to(device, non_blocking=True)
                            kidney_label = labels[:, 2].long().to(device, non_blocking=True)
                            liver_label = labels[:, 3].long().to(device, non_blocking=True)
                            spleen_label = labels[:, 4].long().to(device, non_blocking=True)

                            # 順伝搬（forward）計算(Scalerによる高速化)
                            with torch.set_grad_enabled(phase == 'train'):
                                with autocast():  # float32の保持や勾配の値のスケーリング等を自動的に行うことでTensorCoreを有効活用し、GPU計算の高速化・省メモリ化
                                    outputs = model(inputs)
                                    bowel = outputs[0].squeeze()
                                    extravsation = outputs[1].squeeze()
                                    kidney = outputs[2]
                                    liver = outputs[3]
                                    spleen = outputs[4]
                                    
                                    b_loss = bce_b(bowel, bowel_label)
                                    e_loss = bce_e(extravsation, extravsation_label)
                                    l_loss = cce(liver, liver_label)
                                    k_loss = cce(kidney, kidney_label)
                                    s_loss = cce(spleen, spleen_label)
                                    loss = torch.mean(torch.stack([b_loss, e_loss, l_loss, k_loss, s_loss]))

                                loss = loss / CFG['acc_step']
                                # 訓練時はバックプロパゲーション
                                if phase == 'train':
                                    scaler.scale(loss).backward()
                                    # grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                                    if (i+1)%CFG['acc_step'] == 0:
                                        scaler.step(optimizer)
                                        scaler.update()
                                        optimizer.zero_grad()
                                        
                                progress_bar.set_postfix(loss=str(loss.item()* CFG['acc_step']), refresh=True)
                                # lossの合計を更新
                                epoch_loss += loss.item() * inputs.size(0) * CFG['acc_step']

                                if phase == "val":
                                    bowel_preds.append(bowel.nan_to_num().sigmoid().detach().cpu().numpy())
                                    extravsation_preds.append(extravsation.nan_to_num().sigmoid().detach().cpu().numpy())
                                    kidney_preds.append(kidney.nan_to_num().softmax(1).detach().cpu().numpy())
                                    liver_preds.append(liver.nan_to_num().softmax(1).detach().cpu().numpy())
                                    spleen_preds.append(spleen.nan_to_num().softmax(1).detach().cpu().numpy())
                                    
                                    bowel_labels.append(bowel_label.data.cpu().numpy())
                                    extravsation_labels.append(extravsation_label.data.cpu().numpy())
                                    kidney_labels.append(kidney_label.data.cpu().numpy())
                                    liver_labels.append(liver_label.data.cpu().numpy())
                                    spleen_labels.append(spleen_label.data.cpu().numpy())

                                    bowel_pred_l = np.concatenate([bowel_pred_l, bowel.nan_to_num().sigmoid().detach().cpu().numpy()])
                                    extravsation_pred_l = np.concatenate([extravsation_pred_l, extravsation.nan_to_num().sigmoid().detach().cpu().numpy()])
                                    kidney_list_l.append(kidney.nan_to_num().softmax(1).detach().cpu().numpy())
                                    liver_list_l.append(liver.nan_to_num().softmax(1).detach().cpu().numpy())
                                    spleen_list_l.append(spleen.nan_to_num().softmax(1).detach().cpu().numpy())

                        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                        if phase == 'train':
                            t_loss = epoch_loss
                        else:  
                            v_loss = epoch_loss
                            
                        if phase == 'val':
                            kidney_pred_l = np.concatenate(kidney_list_l)
                            liver_pred_l = np.concatenate(liver_list_l)
                            spleen_pred_l = np.concatenate(spleen_list_l)

                            valid_df = train[train['fold']==fold].iloc[:len(bowel_pred_l)]
                            pred_df = valid_df[['patient_id', 'series_id']].copy().reset_index(drop=True)


                            pred_df['bowel_injury'] = bowel_pred_l
                            pred_df['extravasation_injury'] = extravsation_pred_l
                            kidney_df = pd.DataFrame(kidney_pred_l, columns=['kidney_healthy', 'kidney_low', 'kidney_high'])
                            pred_df = pd.concat([pred_df, kidney_df], axis=1).reset_index(drop=True)

                            liver_df = pd.DataFrame(liver_pred_l, columns=['liver_healthy', 'liver_low', 'liver_high'])
                            pred_df = pd.concat([pred_df, liver_df], axis=1).reset_index(drop=True)
                            spleen_df = pd.DataFrame(spleen_pred_l, columns=['spleen_healthy', 'spleen_low', 'spleen_high'])
                            pred_df = pd.concat([pred_df, spleen_df], axis=1).reset_index(drop=True)

                            # pred_df = pred_df.groupby('patient_id').mean().reset_index()
                            pred_df = pred_df.groupby('patient_id').max().reset_index()

                            pred_df = pred_df.drop(['series_id'], axis=1)
                            pred_df['bowel_healthy'] = 1 - pred_df['bowel_injury']
                            pred_df['extravasation_healthy'] = 1 - pred_df['extravasation_injury']
                            pred_df = pred_df[['patient_id', 'bowel_healthy', 'bowel_injury', 'extravasation_healthy', 'extravasation_injury', 
                                                    'kidney_healthy', 'kidney_low', 'kidney_high', 
                                                    'liver_healthy', 'liver_low', 'liver_high',
                                                    'spleen_healthy', 'spleen_low', 'spleen_high'
                                                    ]]
                            p_df = pred_df.copy()
                            p_df = p_df.drop(['patient_id'], axis=1)

                            sol_df = valid_df.copy()

                            sol_df = sol_df.drop(['InstanceNumber', 'fold', 'series_id'], axis=1)

                            sol_df = sol_df.groupby('patient_id').mean().reset_index()

                            # sol_df['bowel_weight'] = 2.0
                            # sol_df['extravasation_weight'] = 6.0
                            sol_df['bowel_weight'] = sol_df['bowel_healthy'] + sol_df['bowel_injury']*2
                            sol_df['extravasation_weight'] = sol_df['extravasation_healthy'] + sol_df['extravasation_injury']*6
                            sol_df['kidney_weight'] = sol_df['kidney_healthy'] + sol_df['kidney_low']*2 + sol_df['kidney_high']*4
                            sol_df['liver_weight'] = sol_df['liver_healthy'] + sol_df['liver_low']*2 + sol_df['liver_high']*4
                            sol_df['spleen_weight'] = sol_df['spleen_healthy'] + sol_df['spleen_low']*2 + sol_df['spleen_high']*4
                            sol_df['any_injury_weight'] = 6.0

                            sol_df = sol_df.drop(['patient_id'], axis=1)
                            weight_loss = score(sol_df, p_df)
                            
                            bowel_preds = np.concatenate(bowel_preds)
                            extravsation_preds = np.concatenate(extravsation_preds)
                            kidney_preds = np.concatenate(kidney_preds)
                            liver_preds = np.concatenate(liver_preds)
                            spleen_preds = np.concatenate(spleen_preds)
                            
                            bowel_labels = np.concatenate(bowel_labels)
                            extravsation_labels = np.concatenate(extravsation_labels)
                            kidney_labels = np.concatenate(kidney_labels)
                            liver_labels = np.concatenate(liver_labels)
                            spleen_labels = np.concatenate(spleen_labels)

                            
                            kidney_preds = kidney_preds / np.sum(kidney_preds, axis=1, keepdims=True)
                            liver_preds = liver_preds / np.sum(liver_preds, axis=1, keepdims=True)
                            spleen_preds = spleen_preds / np.sum(spleen_preds, axis=1, keepdims=True)

                            bowel_auc = roc_auc_score(bowel_labels, bowel_preds)
                            extravsation_auc = roc_auc_score(extravsation_labels, extravsation_preds)
                            kidney_auc = roc_auc_score(kidney_labels, kidney_preds, multi_class='ovr', average='micro')
                            liver_auc = roc_auc_score(liver_labels, liver_preds, multi_class='ovr', average='micro')
                            spleen_auc = roc_auc_score(spleen_labels, spleen_preds, multi_class='ovr', average='micro')
                            mean_auc = (bowel_auc +extravsation_auc +kidney_auc +liver_auc +spleen_auc) / 5
                            
                        if phase == 'val' and weight_loss < best_v_loss:
                            best_v_loss = weight_loss
                            best_epoch = epoch
                            # 3. 学習によって作られたモデルを保存しましょう
                            torch.save(model.state_dict(),'./cls_model/'+CFG['model']+'_fold_{}'.format(fold))
                    wandb.log({
                        'train_loss' : t_loss,
                        'valid_loss' : v_loss, 
                        'weight_loss' : weight_loss,
                        'bowel_auc' : bowel_auc,
                        'extravsation_auc' : extravsation_auc,
                        'kidney_auc' : kidney_auc,
                        'liver_auc' : liver_auc,
                        'spleen_auc' : spleen_auc, 
                        'mean_auc' : mean_auc,
                        'lr' : optimizer.param_groups[0]["lr"]
                    })
                scheduler.step(epoch+1)
                # scheduler_warmup.step()
                    
                
            # 不要ファイルを削除し、メモリ開放
            del model, optimizer, train_dataloader, val_dataloader, scaler
            torch.cuda.empty_cache()
            print(best_v_loss)

In [27]:
if CFG['debug']:
    training(train.iloc[:1000])
else:
    training(train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shimosaba. Use `wandb login --relogin` to force relogin


Fold 1
-------------------------------------------------


100%|██████████| 585/585 [02:51<00:00,  3.41it/s, loss=0.85179203748703]     


0.4124660552990241


bowel_auc,▁▃▄▆▇▇▇▇█▇▇▇▇▇▇
extravsation_auc,▁▄▄▃▄▅▅▆▆▆▇▇███
kidney_auc,▁▄▄▆▇▆▇▇█▇█████
liver_auc,▁▂▃▅▆▅▅▆▆▇▇████
lr,▁▃▆███▇▇▆▆▅▄▃▃▂
mean_auc,▁▃▄▅▇▇▇██▇▇▇███
spleen_auc,▁▂▂▅▆▆▇▇▆▇▇████
train_loss,█▆▅▅▅▄▄▄▃▃▂▂▂▁▁
valid_loss,█▇▇▅▃▆▃▁▃▂▃▁▂▂▂
weight_loss,▇██▅▃▆▂▂▂▃▁▁▂▁▂
bowel_auc,0.742


...

0.4502458488010385


bowel_auc,▁▁▃▃▆▅▆▇▇█▇████
extravsation_auc,▂▁▂▃▆▆▇█▇▇█▇███
kidney_auc,▁▁▂▃▇▇▇▇▇▇▇████
liver_auc,▁▄▅▆▆▅▆▇▇▇▇▇███
lr,▁▃▆███▇▇▆▆▅▄▃▃▂
mean_auc,▁▁▂▃▆▆▇▇▇██████
spleen_auc,▁▂▂▅▇▇▇▇▇██████
train_loss,█▅▅▅▄▄▄▃▃▃▂▂▂▁▁
valid_loss,██▇▇▄▄▄▂▃▁▁▃▁▃▁
weight_loss,██▇▆▅▄▃▃▃▁▁▃▂▃▁
bowel_auc,0.83278


Fold 3
-------------------------------------------------


100%|██████████| 594/594 [02:55<00:00,  3.38it/s, loss=0.032480716705322266] 


0.401655816342594


In [28]:
np.mean([])

nan

In [29]:
scores = []
for fold in range(4):
    model = RSNAClassifier(CFG['model'], seq_len=CFG['slice_size'], pretrained=True)
    model.load_state_dict(torch.load('cls_model/'+CFG['model']+'_fold_{}'.format(fold)))
    model.eval()
    model.to(device)

    valid_df = train[train['fold']==fold]
    valid_ds = BreastDataset(valid_df,
                            transforms=get_valid_transforms(), 
                            output_label=True)
        
        
    # val_loaderを作成
    val_loader = DataLoader(valid_ds, 
                            batch_size=2, 
                            num_workers=os.cpu_count(), 
                            pin_memory=True, 
                            drop_last=True)

    bowel_pred = np.array([])
    extravsation_pred = np.array([])
    kidney_list = []
    liver_list = []
    spleen_list = []

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(tqdm(val_loader)):
            # GPUが使えるならGPUにデータを送る
            inputs = inputs.to(device)
            inputs = inputs.float()
            bowel_label = labels[:, 0].float().cuda()
            extravsation_label = labels[:, 1].float().cuda()
            kidney_label = labels[:, 2].long().cuda()
            liver_label = labels[:, 3].long().cuda()
            spleen_label = labels[:, 4].long().cuda()

            outputs = model(inputs)
            bowel = outputs[0].squeeze()
            extravsation = outputs[1].squeeze()
            kidney = outputs[2]
            liver = outputs[3]
            spleen = outputs[4]

            bowel_pred = np.concatenate([bowel_pred, bowel.nan_to_num().sigmoid().detach().cpu().numpy()])
            extravsation_pred = np.concatenate([extravsation_pred, extravsation.nan_to_num().sigmoid().detach().cpu().numpy()])
            kidney_list.append(kidney.nan_to_num().softmax(1).detach().cpu().numpy())
            liver_list.append(liver.nan_to_num().softmax(1).detach().cpu().numpy())
            spleen_list.append(spleen.nan_to_num().softmax(1).detach().cpu().numpy())

    del model
    torch.cuda.empty_cache()
    kidney_pred = np.concatenate(kidney_list)
    liver_pred = np.concatenate(liver_list)
    spleen_pred = np.concatenate(spleen_list)

    valid_df = valid_df.iloc[:len(bowel_pred)]
    pred_df = valid_df[['patient_id', 'series_id']].copy().reset_index(drop=True)


    pred_df['bowel_injury'] = bowel_pred
    pred_df['extravasation_injury'] = extravsation_pred
    kidney_df = pd.DataFrame(kidney_pred, columns=['kidney_healthy', 'kidney_low', 'kidney_high'])
    pred_df = pd.concat([pred_df, kidney_df], axis=1).reset_index(drop=True)

    liver_df = pd.DataFrame(liver_pred, columns=['liver_healthy', 'liver_low', 'liver_high'])
    pred_df = pd.concat([pred_df, liver_df], axis=1).reset_index(drop=True)
    spleen_df = pd.DataFrame(spleen_pred, columns=['spleen_healthy', 'spleen_low', 'spleen_high'])
    pred_df = pd.concat([pred_df, spleen_df], axis=1).reset_index(drop=True)

    # pred_df = pred_df.groupby('patient_id').mean().reset_index()
    pred_df = pred_df.groupby('patient_id').max().reset_index()

    pred_df = pred_df.drop(['series_id'], axis=1)
    pred_df['bowel_healthy'] = 1 - pred_df['bowel_injury']
    pred_df['extravasation_healthy'] = 1 - pred_df['extravasation_injury']
    pred_df = pred_df[['patient_id', 'bowel_healthy', 'bowel_injury', 'extravasation_healthy', 'extravasation_injury', 
                            'kidney_healthy', 'kidney_low', 'kidney_high', 
                            'liver_healthy', 'liver_low', 'liver_high',
                            'spleen_healthy', 'spleen_low', 'spleen_high'
                            ]]
    p_df = pred_df.copy()
    p_df = p_df.drop(['patient_id'], axis=1)

    sol_df = valid_df.copy()

    sol_df = sol_df.drop(['InstanceNumber', 'fold', 'series_id'], axis=1)

    sol_df = sol_df.groupby('patient_id').mean().reset_index()

    # sol_df['bowel_weight'] = 2.0
    # sol_df['extravasation_weight'] = 6.0
    sol_df['bowel_weight'] = sol_df['bowel_healthy'] + sol_df['bowel_injury']*2
    sol_df['extravasation_weight'] = sol_df['extravasation_healthy'] + sol_df['extravasation_injury']*6
    sol_df['kidney_weight'] = sol_df['kidney_healthy'] + sol_df['kidney_low']*2 + sol_df['kidney_high']*4
    sol_df['liver_weight'] = sol_df['liver_healthy'] + sol_df['liver_low']*2 + sol_df['liver_high']*4
    sol_df['spleen_weight'] = sol_df['spleen_healthy'] + sol_df['spleen_low']*2 + sol_df['spleen_high']*4
    sol_df['any_injury_weight'] = 6.0

    sol_df = sol_df.drop(['patient_id'], axis=1)
    scores.append(score(sol_df, p_df))

100%|██████████| 594/594 [05:10<00:00,  1.92it/s]


In [30]:
np.mean(scores)

0.4023617482663432

In [31]:
np.mean(scores)

0.4023617482663432

0.4338072706736688